# 🧬 StrandWeaver — K-Weaver + ErrorSmith Training (Colab GPU)

Train the **last 2 AI modules** to complete the StrandWeaver model suite:

| Track | Module | Type | What it learns |
|-------|--------|------|----------------|
| **A** | **K-Weaver** | 4 XGBoost regressors | Predict optimal k-mer sizes (DBG, UL-overlap, extension, polish) from 19 read features |
| **B** | **ErrorSmith** | 1 XGBoost classifier | Classify per-base errors (correct / sub / ins / del / HP-error) from 16 alignment features |

**Track A** is fully self-contained — it synthesises genomes, simulates reads, sweeps k-values, and scores assemblies. No external data needed.

**Track B** uses real CHM13 sequencing data. You can either:
- Auto-download public SRA reads + align *(requires ~50 GB disk, ~2 h)*
- Upload pre-aligned BAMs to Google Drive *(recommended)*

### Setup
1. `Runtime → Change runtime type → T4 GPU` (or A100 if available)
2. Upload this notebook to Colab
3. Run cells top-to-bottom — sections are clearly marked **[Track A]** and **[Track B]**

### Runtime estimates (T4 GPU / High-RAM)
| Config | K-Weaver (Track A) | ErrorSmith (Track B) | Total |
|--------|--------------------|----------------------|-------|
| Quick test | ~10 min (10 genomes) | ~5 min (500k bases) | ~15 min |
| Standard | ~1-2 h (100 genomes) | ~30 min (5M bases) | ~2 h |
| Full | ~3-4 h (200 genomes) | ~1-2 h (20M bases) | ~5 h |

## 1. Environment Setup

In [1]:
# ── Verify GPU ──────────────────────────────────────────────────────
import subprocess, sys, os

result = subprocess.run(
    ['nvidia-smi', '--query-gpu=name,memory.total,driver_version', '--format=csv,noheader'],
    capture_output=True, text=True
)
if result.returncode == 0:
    print(f"✓ GPU detected: {result.stdout.strip()}")
    GPU_AVAILABLE = True
else:
    print("⚠ No GPU detected — XGBoost will use CPU (still fast, just slower HP search)")
    GPU_AVAILABLE = False

# Check RAM
import psutil
ram_gb = psutil.virtual_memory().total / 1e9
print(f"✓ RAM: {ram_gb:.1f} GB {'(High-RAM ✓)' if ram_gb > 20 else '(consider High-RAM runtime)'}")
print(f"✓ Python: {sys.version.split()[0]}")

✓ GPU detected: Tesla T4, 15360 MiB, 580.82.07
✓ RAM: 13.6 GB (consider High-RAM runtime)
✓ Python: 3.12.12


In [ ]:
# ── Install dependencies ────────────────────────────────────────────
!pip install -q xgboost scikit-learn numpy pandas optuna pysam biopython

# Clone StrandWeaver (dev branch) and install — pull latest on re-runs
!git clone -b dev https://github.com/pgrady1322/strandweaver.git /content/strandweaver 2>/dev/null || \
    (cd /content/strandweaver && git pull origin dev)
!cd /content/strandweaver && pip install -e . -q

print("\n✓ Dependencies installed")

Already cloned
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 56.8 MB/s eta 0:00:00
  Building editable for strandweaver (pyproject.toml) ... done

✓ Dependencies installed


In [6]:
# ── Mount Google Drive (for saving results) ─────────────────────────
from google.colab import drive
drive.mount('/content/drive')

# ── Directory layout ────────────────────────────────────────────────
GDRIVE_DIR     = '/content/drive/MyDrive/Colab Notebooks/strandweaver'
KWEAVER_OUT    = '/content/kweaver_training'
ERRORSMITH_OUT = '/content/errorsmith_training'
MODELS_OUT     = '/content/trained_models'

# Final export paths on Drive
GDRIVE_KWEAVER  = os.path.join(GDRIVE_DIR, 'kweaver_models')
GDRIVE_ERRORSMITH = os.path.join(GDRIVE_DIR, 'errorsmith_models')

for d in [GDRIVE_DIR, KWEAVER_OUT, ERRORSMITH_OUT, MODELS_OUT]:
    os.makedirs(d, exist_ok=True)

print(f"✓ Google Drive mounted")
print(f"✓ Working dirs created")
print(f"  K-Weaver output:   {KWEAVER_OUT}")
print(f"  ErrorSmith output:  {ERRORSMITH_OUT}")
print(f"  Models output:      {MODELS_OUT}")
print(f"  Drive export:       {GDRIVE_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Google Drive mounted
✓ Working dirs created
  K-Weaver output:   /content/kweaver_training
  ErrorSmith output:  /content/errorsmith_training
  Models output:      /content/trained_models
  Drive export:       /content/drive/MyDrive/Colab Notebooks/strandweaver


In [7]:
# ── Verify StrandWeaver imports ──────────────────────────────────────
sys.path.insert(0, '/content/strandweaver')

from strandweaver.preprocessing.kweaver_module import (
    FeatureExtractor, ReadFeatures, KmerPrediction, KWeaverPredictor,
)
from strandweaver.io_utils import SeqRead

import xgboost as xgb
import numpy as np
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s', datefmt='%H:%M:%S')
logger = logging.getLogger('colab_training')

print(f"✓ StrandWeaver imports OK")
print(f"✓ XGBoost {xgb.__version__}")
print(f"✓ NumPy {np.__version__}")
print(f"✓ Pandas {pd.__version__}")

✓ StrandWeaver imports OK
✓ XGBoost 3.1.3
✓ NumPy 2.0.2
✓ Pandas 2.2.2


---
# Track A: K-Weaver Training 🔑

**Goal:** Train 4 XGBoost regressors that predict optimal k-mer sizes from read characteristics.

| Model | Predicts | Typical range |
|-------|----------|---------------|
| `dbg_model.pkl` | DBG construction k | 15–127 |
| `ul_overlap_model.pkl` | Ultra-long overlap k | 201–1001 |
| `extension_model.pkl` | Contig extension k | 21–127 |
| `polish_model.pkl` | Polishing k | 31–127 |

**Pipeline:** Synthesise genomes → simulate reads → extract 19 ReadFeatures → sweep k-values → score assemblies → pick best k → train regressors

### 2A. Configuration
Adjust these parameters before running. The **Standard** preset balances quality and runtime (~1-2 h on T4).

In [ ]:
# ══════════════════════════════════════════════════════════════════════
#  K-WEAVER CONFIGURATION — edit these before running
# ══════════════════════════════════════════════════════════════════════

# ── Preset selector ─────────────────────────────────────────────────
# Options: 'quick_test', 'standard', 'full'
#
# Timing estimates (Colab free tier, T4 GPU):
#   quick_test :  ~3 min  ( 10 genomes × 3 techs × 4 k =   120 k-scores)
#   standard   : ~15 min  ( 50 genomes × 3 techs × 6 k =   900 k-scores)
#   full       : ~30 min  (100 genomes × 3 techs × 8 k = 2,400 k-scores)
#
# k-sweep uses a fast k-mer spectrum proxy (~0.2 s/k), NOT full
# DBG assembly (~45 s/k).  Even 'full' fits comfortably in one session.
KWEAVER_PRESET = 'full'

PRESETS = {
    'quick_test': {
        'num_genomes': 10,
        'min_genome_size': 500_000,
        'max_genome_size': 2_000_000,
        'technologies': ['hifi', 'ont', 'illumina'],
        'k_values': [21, 31, 51, 77],
    },
    'standard': {
        'num_genomes': 50,
        'min_genome_size': 500_000,
        'max_genome_size': 5_000_000,
        'technologies': ['hifi', 'ont', 'illumina'],
        'k_values': [15, 21, 31, 51, 77, 101],
    },
    'full': {
        'num_genomes': 100,
        'min_genome_size': 500_000,
        'max_genome_size': 5_000_000,
        'technologies': ['hifi', 'ont', 'illumina'],
        'k_values': [15, 21, 31, 41, 51, 63, 77, 101],
    },
}

kw_config = PRESETS[KWEAVER_PRESET]
SEED = 42

print(f"✓ K-Weaver preset: {KWEAVER_PRESET}")
print(f"  Genomes:      {kw_config['num_genomes']}")
print(f"  Genome range: {kw_config['min_genome_size']:,} – {kw_config['max_genome_size']:,} bp")
print(f"  Technologies: {kw_config['technologies']}")
print(f"  K sweep:      {kw_config['k_values']}")
print(f"  Total samples: {kw_config['num_genomes'] * len(kw_config['technologies'])}")

✓ K-Weaver preset: full
  Genomes:      200
  Genome range: 500,000 – 20,000,000 bp
  Technologies: ['hifi', 'ont', 'illumina']
  K sweep:      [15, 17, 21, 25, 31, 41, 51, 63, 77, 91, 101, 127]
  Total samples: 600


### 3A. Generate K-Weaver Training Data
This cell runs the full data generation pipeline. Progress is logged per-genome.

In [10]:
# ── Generate K-Weaver training CSV ───────────────────────────────────
# This calls the generate_kweaver_training_data.py script inline
# so we can monitor progress in the notebook output.

sys.path.insert(0, '/content/strandweaver/strandweaver/training/scripts/')
from generate_kweaver_training_data import generate_kweaver_training_data
from pathlib import Path

import time
t0 = time.time()

kweaver_csv = generate_kweaver_training_data(
    output_dir=Path(KWEAVER_OUT),
    num_genomes=kw_config['num_genomes'],
    min_genome_size=kw_config['min_genome_size'],
    max_genome_size=kw_config['max_genome_size'],
    technologies=kw_config['technologies'],
    k_values=kw_config['k_values'],
    seed=SEED,
)

elapsed = time.time() - t0
print(f"\n✅ K-Weaver training CSV: {kweaver_csv}")
print(f"   Elapsed: {elapsed/60:.1f} min")

KeyboardInterrupt: 

In [ ]:
# ── Inspect the generated data ───────────────────────────────────────
df_kw = pd.read_csv(kweaver_csv)
print(f"Shape: {df_kw.shape}")
print(f"\nTarget distribution:")
for col in ['best_dbg_k', 'best_ul_k', 'best_extension_k', 'best_polish_k']:
    print(f"  {col}: mean={df_kw[col].mean():.1f}, std={df_kw[col].std():.1f}, "
          f"range=[{df_kw[col].min()}, {df_kw[col].max()}]")

print(f"\nTechnology breakdown:")
print(df_kw['technology'].value_counts().to_string())

print(f"\nFeature summary (first 5 rows):")
feature_cols = [
    'mean_read_length', 'median_read_length', 'read_length_n50',
    'mean_base_quality', 'estimated_error_rate', 'gc_content',
    'read_type_encoded', 'num_reads',
]
df_kw[feature_cols].head()

### 4A. Train K-Weaver XGBoost Regressors
Trains 4 models with **Optuna Bayesian hyperparameter search** (20 trials each) + 5-fold CV on the best config.

In [ ]:
# ── K-Weaver training with Optuna HP search ─────────────────────────
import optuna
import pickle
import json
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

optuna.logging.set_verbosity(optuna.logging.WARNING)

# 19 ReadFeature columns (input features)
KWEAVER_FEATURES = [
    'mean_read_length', 'median_read_length', 'read_length_n50',
    'min_read_length', 'max_read_length', 'read_length_std',
    'mean_base_quality', 'median_base_quality', 'estimated_error_rate',
    'total_bases', 'num_reads',
    'estimated_genome_size', 'estimated_coverage',
    'gc_content', 'gc_std',
    'read_type_encoded', 'is_paired_end',
    'kmer_spectrum_peak', 'kmer_diversity',
]

# 4 target columns (one model each)
KWEAVER_TARGETS = {
    'dbg_model':       'best_dbg_k',
    'ul_overlap_model': 'best_ul_k',
    'extension_model':  'best_extension_k',
    'polish_model':     'best_polish_k',
}

N_OPTUNA_TRIALS = 20
N_CV_FOLDS = 5

# XGBoost device config
XGB_DEVICE = 'cuda' if GPU_AVAILABLE else 'cpu'
XGB_TREE = 'hist'  # gpu_hist is deprecated in XGBoost 2.x — 'hist' auto-uses GPU


def optuna_objective(trial, X, y):
    """Optuna objective for XGBoost regression HP search."""
    params = {
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 800),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'gamma': trial.suggest_float('gamma', 0, 5.0),
        'tree_method': XGB_TREE,
        'device': XGB_DEVICE,
        'random_state': SEED,
    }

    model = xgb.XGBRegressor(**params)
    kf = KFold(n_splits=3, shuffle=True, random_state=SEED)
    scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
    return scores.mean()


# ── Prepare data ────────────────────────────────────────────────────
df_kw = pd.read_csv(kweaver_csv)

# Fill NaN in optional features (genome_size, coverage, kmer_spectrum, kmer_diversity)
df_kw[KWEAVER_FEATURES] = df_kw[KWEAVER_FEATURES].fillna(0)

X_all = df_kw[KWEAVER_FEATURES].values
print(f"Training data: {X_all.shape[0]} samples × {X_all.shape[1]} features\n")

# ── Train each model ────────────────────────────────────────────────
kweaver_results = {}
kweaver_save_dir = os.path.join(MODELS_OUT, 'kweaver')
os.makedirs(kweaver_save_dir, exist_ok=True)

for model_name, target_col in KWEAVER_TARGETS.items():
    print(f"{'═'*60}")
    print(f"  Training: {model_name} → {target_col}")
    print(f"{'═'*60}")

    y = df_kw[target_col].values

    # ── Optuna HP search ────────────────────────────────────────
    print(f"  Running Optuna ({N_OPTUNA_TRIALS} trials)...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: optuna_objective(trial, X_all, y),
                   n_trials=N_OPTUNA_TRIALS, show_progress_bar=True)

    best_params = study.best_params
    best_params['tree_method'] = XGB_TREE
    best_params['device'] = XGB_DEVICE
    best_params['random_state'] = SEED

    print(f"  Best MSE (neg): {study.best_value:.4f}")
    print(f"  Best params: depth={best_params['max_depth']}, "
          f"lr={best_params['learning_rate']:.4f}, "
          f"n_est={best_params['n_estimators']}")

    # ── 5-fold CV with best params ──────────────────────────────
    print(f"  Running {N_CV_FOLDS}-fold CV with best params...")
    model = xgb.XGBRegressor(**best_params)
    kf = KFold(n_splits=N_CV_FOLDS, shuffle=True, random_state=SEED)

    cv_mse = []
    cv_r2 = []
    cv_mae = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_all)):
        X_train, X_val = X_all[train_idx], X_all[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        y_pred = model.predict(X_val)

        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)

        cv_mse.append(mse)
        cv_r2.append(r2)
        cv_mae.append(mae)
        print(f"    Fold {fold+1}: MSE={mse:.2f}, MAE={mae:.2f}, R²={r2:.4f}")

    print(f"  CV Summary: MSE={np.mean(cv_mse):.2f}±{np.std(cv_mse):.2f}, "
          f"R²={np.mean(cv_r2):.4f}±{np.std(cv_r2):.4f}, "
          f"MAE={np.mean(cv_mae):.2f}")

    # ── Final model (train on all data) ─────────────────────────
    final_model = xgb.XGBRegressor(**best_params)
    final_model.fit(X_all, y)

    # Save
    pkl_path = os.path.join(kweaver_save_dir, f'{model_name}.pkl')
    with open(pkl_path, 'wb') as f:
        pickle.dump(final_model, f)
    print(f"  ✓ Saved: {pkl_path}")

    kweaver_results[model_name] = {
        'target': target_col,
        'best_params': best_params,
        'cv_mse_mean': float(np.mean(cv_mse)),
        'cv_mse_std': float(np.std(cv_mse)),
        'cv_r2_mean': float(np.mean(cv_r2)),
        'cv_r2_std': float(np.std(cv_r2)),
        'cv_mae_mean': float(np.mean(cv_mae)),
        'cv_mae_std': float(np.std(cv_mae)),
        'n_samples': len(y),
    }
    print()

# Save results summary
results_path = os.path.join(kweaver_save_dir, 'training_results.json')
with open(results_path, 'w') as f:
    json.dump(kweaver_results, f, indent=2, default=str)

print(f"\n{'═'*60}")
print(f"  K-Weaver training complete!")
print(f"  Models saved to: {kweaver_save_dir}")
print(f"{'═'*60}")

### 5A. Validate K-Weaver Models
Quick sanity check: load the saved models and run inference on a synthetic sample.

In [ ]:
# ── Validate: load models and run sample predictions ─────────────────
print("Loading trained K-Weaver models for validation...\n")

# Load all 4 models
kw_models = {}
for model_name in KWEAVER_TARGETS:
    pkl_path = os.path.join(kweaver_save_dir, f'{model_name}.pkl')
    with open(pkl_path, 'rb') as f:
        kw_models[model_name] = pickle.load(f)
    print(f"  ✓ Loaded {model_name}")

# UL applicability threshold (must match kweaver_module.UL_MIN_N50)
UL_MIN_N50 = 50_000

# Test predictions on 3 representative samples from training data
print(f"\n{'─'*70}")
print(f"{'Technology':<12} {'DBG k':>8} {'UL k':>8} {'Ext k':>8} {'Polish k':>8}  UL status")
print(f"{'─'*70}")

test_samples = df_kw.groupby('technology').first().reset_index()
for _, row in test_samples.iterrows():
    x = row[KWEAVER_FEATURES].values.reshape(1, -1).astype(np.float64)
    preds = {name: int(round(model.predict(x)[0])) for name, model in kw_models.items()}
    tech = row['technology']

    # Determine UL applicability from read_length_n50 feature
    n50 = row.get('read_length_n50', 0)
    if n50 >= UL_MIN_N50:
        ul_status = "✓ applicable"
    else:
        ul_status = f"⚠ advisory (N50={n50:,.0f} bp < 50 Kb)"

    print(f"{tech:<12} {preds['dbg_model']:>8} {preds['ul_overlap_model']:>8} "
          f"{preds['extension_model']:>8} {preds['polish_model']:>8}  {ul_status}")

print(f"{'─'*70}")
print(f"\n✓ K-Weaver validation complete — predictions look reasonable!")
print(f"\n  ℹ UL overlap k is only reliable for ultra-long reads (N50 ≥ 50 Kb).")
print(f"    Reads with shorter N50 should not be fed into the UL overlap slot;"  )
print(f"    doing so is likely to produce mis-joins and fragmented scaffolds.")

---
# Track B: ErrorSmith Training 🔬

**Goal:** Train 1 XGBoost multiclass classifier for per-base error type prediction.

| Class | Label | Description |
|-------|-------|-------------|
| 0 | `correct` | Base matches reference |
| 1 | `substitution` | Wrong base |
| 2 | `insertion` | Extra base in read |
| 3 | `deletion` | Missing base in read |
| 4 | `homopolymer_error` | Ins/del in homopolymer context |

**Data source:** Real CHM13 T2T alignments (public SRA data)

**Two options:**
- **Option 1 (recommended):** Upload pre-aligned BAMs to Google Drive
- **Option 2:** Auto-download from SRA *(requires sra-toolkit, minimap2, samtools — ~50 GB disk, ~2 h)*

### 2B. Configuration & Data Source

**Edit the cell below to match your setup.**

If uploading BAMs to Google Drive, place them at:
```
My Drive/Colab Notebooks/strandweaver/chm13_hifi.sorted.bam    (+ .bai)
My Drive/Colab Notebooks/strandweaver/chm13_ont.sorted.bam     (+ .bai)
My Drive/Colab Notebooks/strandweaver/chm13_illumina.sorted.bam (+ .bai)
```

If using auto-download, the script will fetch reads from SRA and align them.

In [ ]:
# ══════════════════════════════════════════════════════════════════════
#  ERRORSMITH CONFIGURATION — edit these before running
# ══════════════════════════════════════════════════════════════════════

# ── Data source mode ────────────────────────────────────────────────
# 'drive'    : Use BAMs uploaded to Google Drive (recommended)
# 'download' : Auto-download from SRA + align (requires disk space + tools)
ERRORSMITH_MODE = 'download'

# ── BAM paths (for 'drive' mode) ────────────────────────────────────
# Set to None for any technology you don't have
HIFI_BAM     = os.path.join(GDRIVE_DIR, 'chm13_hifi.sorted.bam')
ONT_BAM      = os.path.join(GDRIVE_DIR, 'chm13_ont.sorted.bam')
ILLUMINA_BAM = os.path.join(GDRIVE_DIR, 'chm13_illumina.sorted.bam')

# ── Reference genome ────────────────────────────────────────────────
# Will auto-download CHM13v2.0 if not found
REFERENCE_PATH = os.path.join(GDRIVE_DIR, 'chm13v2.0.fa')
AUTO_DOWNLOAD_REF = True  # Download if not found

# ── Subsampling ─────────────────────────────────────────────────────
# Bases per technology — higher = more training data but slower
ERRORSMITH_SUBSAMPLE = 5_000_000  # 5M bases/tech → ~300k error rows total

# ── Chromosomes to process ──────────────────────────────────────────
# Default: chr1-chr5 (good coverage, manageable runtime)
ERRORSMITH_CHROMS = [f'chr{i}' for i in range(1, 6)]

# ── Threads ─────────────────────────────────────────────────────────
ERRORSMITH_THREADS = 4

# ── Validate BAMs exist ────────────────────────────────────────────
if ERRORSMITH_MODE == 'drive':
    bam_status = {}
    for name, path in [('HiFi', HIFI_BAM), ('ONT', ONT_BAM), ('Illumina', ILLUMINA_BAM)]:
        exists = os.path.exists(path) if path else False
        bam_status[name] = exists
        status = '✓' if exists else '✗ not found'
        print(f"  {name}: {status}  ({path})")

    available_techs = sum(bam_status.values())
    if available_techs == 0:
        print(f"\n⚠ No BAMs found! Either:")
        print(f"  1. Upload BAMs to {GDRIVE_DIR}")
        print(f"  2. Set ERRORSMITH_MODE = 'download'")
    else:
        print(f"\n✓ {available_techs}/3 technologies available")
else:
    print(f"✓ Mode: auto-download from SRA")
    print(f"  This will download ~50 GB of data and requires:")
    print(f"  sra-toolkit, minimap2, samtools")

print(f"\n  Subsample:    {ERRORSMITH_SUBSAMPLE:,} bases/tech")
print(f"  Chromosomes:  {ERRORSMITH_CHROMS}")
print(f"  Reference:    {REFERENCE_PATH}")

### 3B. Download Reference + Install Tools (if needed)

In [ ]:
# ── Download CHM13v2.0 reference if needed ───────────────────────────
from generate_errorsmith_training_data import download_reference

if not os.path.exists(REFERENCE_PATH) and AUTO_DOWNLOAD_REF:
    print("Downloading CHM13v2.0 reference (800 MB)...")
    ref_dir = os.path.dirname(REFERENCE_PATH)
    downloaded_ref = download_reference(Path(ref_dir))
    REFERENCE_PATH = str(downloaded_ref)
    print(f"✓ Reference downloaded: {REFERENCE_PATH}")
elif os.path.exists(REFERENCE_PATH):
    print(f"✓ Reference found: {REFERENCE_PATH}")
else:
    print(f"⚠ Reference not found at {REFERENCE_PATH}")
    print(f"  Set AUTO_DOWNLOAD_REF = True to download automatically")

# ── Install alignment tools for 'download' mode ─────────────────────
if ERRORSMITH_MODE == 'download':
    print("\nInstalling sra-toolkit, minimap2, samtools...")
    !apt-get install -qq -y sra-toolkit samtools
    !pip install -q minimap2
    # Verify
    !which prefetch && echo "✓ sra-toolkit" || echo "✗ sra-toolkit missing"
    !which minimap2 && echo "✓ minimap2" || echo "✗ minimap2 missing"
    !which samtools && echo "✓ samtools" || echo "✗ samtools missing"

### 4B. Generate ErrorSmith Training Data
Processes CHM13 BAMs, parses CIGAR alignments, extracts per-base error features.

In [ ]:
# ── Generate ErrorSmith training CSV ──────────────────────────────────
from generate_errorsmith_training_data import generate_errorsmith_training_data

# Build BAM args based on what's available
es_kwargs = {
    'output_dir': Path(ERRORSMITH_OUT),
    'reference_path': REFERENCE_PATH,
    'subsample': ERRORSMITH_SUBSAMPLE,
    'chroms': ERRORSMITH_CHROMS,
    'threads': ERRORSMITH_THREADS,
    'seed': SEED,
}

if ERRORSMITH_MODE == 'download':
    es_kwargs['download'] = True
else:
    # Only pass BAMs that exist
    if os.path.exists(HIFI_BAM):
        es_kwargs['hifi_bam'] = HIFI_BAM
    if os.path.exists(ONT_BAM):
        es_kwargs['ont_bam'] = ONT_BAM
    if os.path.exists(ILLUMINA_BAM):
        es_kwargs['illumina_bam'] = ILLUMINA_BAM

print("Starting ErrorSmith data generation...")
print(f"  Mode: {ERRORSMITH_MODE}")
print(f"  BAMs: {[k for k in ['hifi_bam','ont_bam','illumina_bam'] if k in es_kwargs]}")
print()

t0 = time.time()
errorsmith_csv = generate_errorsmith_training_data(**es_kwargs)
elapsed = time.time() - t0

print(f"\n✅ ErrorSmith training CSV: {errorsmith_csv}")
print(f"   Elapsed: {elapsed/60:.1f} min")

In [ ]:
# ── Inspect ErrorSmith data ──────────────────────────────────────────
df_es = pd.read_csv(errorsmith_csv)
print(f"Shape: {df_es.shape}")

# Class distribution
ERROR_TYPE_NAMES = {0: 'correct', 1: 'substitution', 2: 'insertion', 3: 'deletion', 4: 'homopolymer_error'}
print(f"\nClass distribution:")
for label, count in df_es['error_type'].value_counts().sort_index().items():
    pct = count / len(df_es) * 100
    name = ERROR_TYPE_NAMES.get(label, f'unknown_{label}')
    print(f"  {label} ({name:20s}): {count:>10,} ({pct:5.1f}%)")

print(f"\nTechnology breakdown:")
print(df_es['technology'].value_counts().to_string())

print(f"\nFeature summary:")
feature_cols = ['base_quality', 'mean_quality_window_5', 'position_in_read',
                'gc_content_local', 'homopolymer_length', 'read_length']
df_es[feature_cols].describe().round(3)

### 5B. Train ErrorSmith Classifier
XGBoost multiclass classifier with **Optuna HP search** + **hybrid resampling** (undersample majority `correct` class, oversample minority error types).

In [ ]:
# ── ErrorSmith training with Optuna + hybrid resampling ──────────────
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, confusion_matrix
)
from sklearn.utils import resample
from collections import Counter

# 16 error features
ERRORSMITH_FEATURES = [
    'base_quality', 'mean_quality_window_5', 'mean_quality_window_20',
    'position_in_read', 'read_length',
    'gc_content_local', 'gc_content_read',
    'homopolymer_length', 'homopolymer_base', 'distance_to_hp',
    'trinucleotide_context', 'pentanucleotide_context',
    'technology_encoded',
    'ref_gc_window_50', 'ref_repeat_flag', 'ref_homopolymer_length',
]

ES_LABEL = 'error_type'
ES_N_OPTUNA_TRIALS = 25
ES_N_CV_FOLDS = 5


def hybrid_resample(X, y, max_majority=100_000, random_state=42):
    """
    Hybrid resampling: undersample majority to max_majority,
    oversample minorities to median class size.
    """
    classes, counts = np.unique(y, return_counts=True)
    class_counts = dict(zip(classes, counts))

    # Cap majority class
    capped = {c: min(n, max_majority) for c, n in class_counts.items()}
    median_size = int(np.median(list(capped.values())))

    X_resampled, y_resampled = [], []
    for cls in classes:
        mask = y == cls
        X_cls, y_cls = X[mask], y[mask]
        target = max(median_size, min(len(X_cls), max_majority))

        if len(X_cls) > target:
            X_rs, y_rs = resample(X_cls, y_cls, n_samples=target,
                                  random_state=random_state, replace=False)
        elif len(X_cls) < target:
            X_rs, y_rs = resample(X_cls, y_cls, n_samples=target,
                                  random_state=random_state, replace=True)
        else:
            X_rs, y_rs = X_cls, y_cls

        X_resampled.append(X_rs)
        y_resampled.append(y_rs)

    return np.vstack(X_resampled), np.concatenate(y_resampled)


def es_optuna_objective(trial, X, y):
    """Optuna objective for ErrorSmith multiclass classifier."""
    params = {
        'max_depth': trial.suggest_int('max_depth', 5, 14),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'gamma': trial.suggest_float('gamma', 0, 5.0),
        'tree_method': XGB_TREE,
        'device': XGB_DEVICE,
        'objective': 'multi:softprob',
        'num_class': 5,
        'eval_metric': 'mlogloss',
        'random_state': SEED,
    }

    model = xgb.XGBClassifier(**params)
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
    scores = cross_val_score(model, X, y, cv=skf, scoring='f1_macro')
    return scores.mean()


# ── Prepare data ────────────────────────────────────────────────────
df_es = pd.read_csv(errorsmith_csv)
X_es = df_es[ERRORSMITH_FEATURES].fillna(0).values
y_es = df_es[ES_LABEL].values

print(f"Raw data: {X_es.shape[0]:,} samples × {X_es.shape[1]} features")
print(f"Class distribution: {dict(Counter(y_es))}")

# ── Hybrid resampling ──────────────────────────────────────────────
X_balanced, y_balanced = hybrid_resample(X_es, y_es, max_majority=100_000)
print(f"\nAfter hybrid resampling: {X_balanced.shape[0]:,} samples")
print(f"Balanced distribution:  {dict(Counter(y_balanced))}")

# ── Optuna HP search ────────────────────────────────────────────────
print(f"\nRunning Optuna ({ES_N_OPTUNA_TRIALS} trials)...")
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: es_optuna_objective(trial, X_balanced, y_balanced),
               n_trials=ES_N_OPTUNA_TRIALS, show_progress_bar=True)

best_params = study.best_params
best_params['tree_method'] = XGB_TREE
best_params['device'] = XGB_DEVICE
best_params['objective'] = 'multi:softprob'
best_params['num_class'] = 5
best_params['eval_metric'] = 'mlogloss'
best_params['random_state'] = SEED

print(f"\nBest F1-macro: {study.best_value:.4f}")
print(f"Best params: depth={best_params['max_depth']}, "
      f"lr={best_params['learning_rate']:.4f}, "
      f"n_est={best_params['n_estimators']}")

# ── 5-fold CV with best params ──────────────────────────────────────
print(f"\nRunning {ES_N_CV_FOLDS}-fold CV...")
skf = StratifiedKFold(n_splits=ES_N_CV_FOLDS, shuffle=True, random_state=SEED)

cv_acc = []
cv_f1 = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_balanced, y_balanced)):
    X_train, X_val = X_balanced[train_idx], X_balanced[val_idx]
    y_train, y_val = y_balanced[train_idx], y_balanced[val_idx]

    model = xgb.XGBClassifier(**best_params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = model.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='macro')
    cv_acc.append(acc)
    cv_f1.append(f1)
    print(f"  Fold {fold+1}: Accuracy={acc:.4f}, F1-macro={f1:.4f}")

print(f"\nCV Summary: Accuracy={np.mean(cv_acc):.4f}±{np.std(cv_acc):.4f}, "
      f"F1-macro={np.mean(cv_f1):.4f}±{np.std(cv_f1):.4f}")

# ── Final model (train on all balanced data) ────────────────────────
print(f"\nTraining final model on all {len(y_balanced):,} samples...")
final_es_model = xgb.XGBClassifier(**best_params)
final_es_model.fit(X_balanced, y_balanced)

# Save
errorsmith_save_dir = os.path.join(MODELS_OUT, 'errorsmith')
os.makedirs(errorsmith_save_dir, exist_ok=True)

pkl_path = os.path.join(errorsmith_save_dir, 'error_classifier.pkl')
with open(pkl_path, 'wb') as f:
    pickle.dump(final_es_model, f)

print(f"\n✓ Saved: {pkl_path}")

# ── Classification report on held-out fold ──────────────────────────
print(f"\n{'═'*60}")
print("Classification Report (last CV fold):")
print(f"{'═'*60}")
class_names = ['correct', 'substitution', 'insertion', 'deletion', 'HP_error']
print(classification_report(y_val, y_pred, target_names=class_names, digits=3))

# Save results
errorsmith_results = {
    'best_params': best_params,
    'cv_accuracy_mean': float(np.mean(cv_acc)),
    'cv_accuracy_std': float(np.std(cv_acc)),
    'cv_f1_macro_mean': float(np.mean(cv_f1)),
    'cv_f1_macro_std': float(np.std(cv_f1)),
    'n_samples_raw': int(len(y_es)),
    'n_samples_balanced': int(len(y_balanced)),
    'class_distribution': dict(Counter(int(c) for c in y_balanced)),
}
with open(os.path.join(errorsmith_save_dir, 'training_results.json'), 'w') as f:
    json.dump(errorsmith_results, f, indent=2, default=str)

print(f"\n{'═'*60}")
print(f"  ErrorSmith training complete!")
print(f"  Model saved to: {pkl_path}")
print(f"{'═'*60}")

---
## 6. Feature Importance Plots

In [ ]:
# ── Feature importance plots ─────────────────────────────────────────
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Feature Importance — K-Weaver + ErrorSmith', fontsize=16, fontweight='bold')

# K-Weaver models (top row + bottom-left)
for idx, (model_name, target_col) in enumerate(KWEAVER_TARGETS.items()):
    ax = axes[idx // 3][idx % 3]
    model = kw_models[model_name]
    importances = model.feature_importances_
    sorted_idx = np.argsort(importances)[-10:]  # Top 10
    ax.barh(range(10), importances[sorted_idx], color='steelblue')
    ax.set_yticks(range(10))
    ax.set_yticklabels([KWEAVER_FEATURES[i] for i in sorted_idx], fontsize=9)
    ax.set_title(f'K-Weaver: {model_name}', fontsize=11)
    ax.set_xlabel('Importance')

# ErrorSmith (bottom-right)
ax = axes[1][2]
es_importances = final_es_model.feature_importances_
sorted_idx = np.argsort(es_importances)[-10:]
ax.barh(range(10), es_importances[sorted_idx], color='coral')
ax.set_yticks(range(10))
ax.set_yticklabels([ERRORSMITH_FEATURES[i] for i in sorted_idx], fontsize=9)
ax.set_title('ErrorSmith: error_classifier', fontsize=11)
ax.set_xlabel('Importance')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig(os.path.join(MODELS_OUT, 'feature_importance.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✓ Feature importance plot saved")

## 7. Export Models to Google Drive
Package all trained models into a tarball and copy to Google Drive for download.

In [ ]:
# ── Package and export to Google Drive ────────────────────────────────
import tarfile, shutil, glob

GDRIVE_TARBALL = os.path.join(GDRIVE_DIR, 'kweaver_errorsmith_models.tar.gz')

# List all model files
model_files = glob.glob(f'{MODELS_OUT}/**/*', recursive=True)
print(f"Model files to export ({len(model_files)}):")
for f in sorted(model_files):
    if os.path.isfile(f):
        size_kb = os.path.getsize(f) / 1024
        print(f"  {os.path.relpath(f, MODELS_OUT):40s} {size_kb:>8.1f} KB")

# Create tarball
print(f"\nCreating tarball...")
with tarfile.open(GDRIVE_TARBALL, 'w:gz') as tar:
    tar.add(MODELS_OUT, arcname='trained_models')

tarball_size = os.path.getsize(GDRIVE_TARBALL) / (1024 * 1024)
print(f"\n✅ Exported to Google Drive:")
print(f"   {GDRIVE_TARBALL}")
print(f"   Size: {tarball_size:.1f} MB")

# Also copy individual model dirs to Drive for easy access
for subdir in ['kweaver', 'errorsmith']:
    src = os.path.join(MODELS_OUT, subdir)
    dst = os.path.join(GDRIVE_DIR, f'{subdir}_models')
    if os.path.isdir(src):
        if os.path.exists(dst):
            shutil.rmtree(dst)
        shutil.copytree(src, dst)
        print(f"   Copied {subdir}/ → {dst}")

print(f"\n📥 To download locally:")
print(f"   1. Open Google Drive → My Drive/Colab Notebooks/strandweaver/")
print(f"   2. Download kweaver_errorsmith_models.tar.gz")
print(f"   3. Extract to strandweaver/trained_models/")

## 8. Summary & Next Steps

In [ ]:
# ── Final summary ────────────────────────────────────────────────────
print("╔══════════════════════════════════════════════════════════════╗")
print("║          StrandWeaver Model Training — Summary              ║")
print("╠══════════════════════════════════════════════════════════════╣")
print("║                                                              ║")
print("║  TRACK A: K-Weaver (4 regressors)                           ║")
print("║  ─────────────────────────────────                           ║")
for name, res in kweaver_results.items():
    print(f"║    {name:20s}  R²={res['cv_r2_mean']:.3f}±{res['cv_r2_std']:.3f}  "
          f"MAE={res['cv_mae_mean']:.1f}  ║")
print("║                                                              ║")
print("║  TRACK B: ErrorSmith (1 classifier)                         ║")
print("║  ──────────────────────────────────                          ║")
print(f"║    error_classifier       F1={errorsmith_results['cv_f1_macro_mean']:.3f}±"
      f"{errorsmith_results['cv_f1_macro_std']:.3f}  "
      f"Acc={errorsmith_results['cv_accuracy_mean']:.3f}  ║")
print("║                                                              ║")
print("║  Output files on Google Drive:                               ║")
print(f"║    {GDRIVE_DIR}/")
print(f"║    ├── kweaver_models/       (4 .pkl files)")
print(f"║    ├── errorsmith_models/    (1 .pkl file)")
print(f"║    └── kweaver_errorsmith_models.tar.gz")
print("║                                                              ║")
print("║  Next steps:                                                 ║")
print("║    1. Download models to local strandweaver/trained_models/  ║")
print("║    2. Run: strandweaver core-assemble --use-ai <reads>       ║")
print("║    3. All 7/7 AI modules now have trained weights! 🎉        ║")
print("║                                                              ║")
print("╚══════════════════════════════════════════════════════════════╝")